In [1]:
from ipynb.fs.defs.functions import htmltotext, some_actions, query_url, send_query, save_data, check_new_articles

In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd

from nltk import sent_tokenize

import json
import time
import random
import traceback
import pickle
import random

In [ ]:
browser = webdriver.Chrome(  executable_path = "C:\\Chrome\\chromedriver" )
browser.set_window_size(1920, 1080)
action  = webdriver.ActionChains(browser)

### Page to check robots

In [ ]:
i = 0
browser.set_page_load_timeout(0.55)

while i != 1000:
    i += 1
    try:
        browser.get("https://www.bloomberg.com/")
        browser.delete_all_cookies()
    except:
        pass

# Confirm not a robot

In [6]:
import time

def click_and_hold( clickToX, clickToY, sleep, reset=1 ):
    """ Click and hold several seconds using X and Y coordinates
    
        Args:
            clickToX ([float]) : [x coordinate]
            clickToY ([float]) : [y coordinate]
            sleep    ([float]) : [hold seconds]
            reset    ([int])   : [1 if want to move mouse, -1 if return to default place]
    """

    # if reset is -1, move mouse to default position
    clickToX *= reset
    clickToY *= reset
    
    if reset == 1:
        action.move_by_offset( clickToX, clickToY)
        action.click_and_hold().perform()
        time.sleep(sleep)
    else:
        action.move_by_offset( clickToX, clickToY)
        
    # delete all actions    
    action.reset_actions()

def confirm_not_robot():
    
    # coordinates of robot button    
    clickToX = browser.get_window_size()['width']/2 - 150 + random.uniform(-2,2)
    clickToY = 250 + random.randint(-1,3) + random.uniform(-2,2)
    click_wait_time = 5 + random.uniform(-0.5,0.5)
    
    # click and hold the robot button
    click_and_hold( clickToX, clickToY, click_wait_time, 1 )
    click_and_hold( clickToX, clickToY, 0, -1)
    time.sleep(click_wait_time)

In [7]:
def process_not_robot( ):
    cnt_capthca = 0
    cnt_except  = 0
    cnt_need    = 3
    
    # return if we confirm 3 times and pange still exist 
    while cnt_capthca != cnt_need and cnt_except != cnt_need :
        print( cnt_capthca, cnt_except )
        try:
            # chekc if confirming captcha in page                
            if browser.find_element_by_id( "px-captcha" ):
                cnt_capthca += 1
                confirm_not_robot()
        except Exception as e:
            cnt_except += 1
            if "no such element" not in str(e):
                print( str(e) )
                
      
    if cnt_capthca == cnt_need :
        browser.delete_all_cookies()

In [8]:
def change_browser():
    # if we still robot change browser
    global browser, action
    browser.close()
    browser = webdriver.Chrome(  executable_path = "C:\\Chrome\\chromedriver" )
    browser.set_window_size(1920, 1080)
    action  = webdriver.ActionChains(browser)

# Process

In [18]:
def get_article_urls_from_html( articles ):
    """ Get titles and hrefs to articles
    
        Args: 
            articles ([string]) : [html string with articles hrefs and titles]
            
        Return:
            [dict] : [dict of articles (url:title)]
    """

    articles_url_name = {}
    
    
    for article in articles[:5]:
        
        # remove the trend position from href ( http://bloomberg.com/article/14?13 )
        if "?" in article['href']:
            article['href'] = article['href'].split("?")[0]
            
        href = source_url + article['href']
        articles_url_name[ href ] = article.get_text()
        
    return articles_url_name

In [10]:
def get_urls_from_category( url, cnt=0 ):
    """ Collect articles urls using page url
        
        Args:
            url ([string]) : [the url of the page]
            cnt ([int, default 0]) : [count of recursion calls]
            
        Return:
            [dict] : [dict of articles (url:title)]
    """

    # getting source html of the page
    html = query_url(url, browser)
    soup = BeautifulSoup(html, 'html.parser')
    
    # if this element in page, all right ( html atribute with articles info )
    compare_element = soup.find_all( "a", attrs={"class" : "story-package-module__story__headline-link"} )
    
    # if articles doesn't exist try to confirm that not a robot   
    if  len( compare_element ) == 0 and cnt != 1 :
        process_not_robot()
        return get_urls_from_category( url, cnt=1 )
    # if we already try to confirm change browser    
    elif len( compare_element ) == 0 and cnt == 1:
        change_browser()
        return {}
    
    articles = compare_element
    
    return get_article_urls_from_html( articles )

In [11]:
def get_article_content( url, cnt=0 ):
    """ Collect article content by url
    
        Args:
            url ([string]) : [article url]
            
        Return:
            [string article content, string article time]
    """
    
    # geting sourse html of the page
    html = query_url(url, browser)
    soup = BeautifulSoup(html, 'html.parser')
    
    # if this element in page, all right    
    compare_element = soup.find_all( attrs={"class" : "body-copy-v2"} )
    
    # if articles doesn't exist try to confirm that not a robot   
    if  len( compare_element ) == 0 and cnt != 1 :
        process_not_robot()
        return get_article_content( url, cnt=1 )
    # if we already try to confirm change browser 
    elif len( compare_element ) == 0 and cnt == 1:
        change_browser()
        return "", ""
    
    # get free content ( pre_wiev), and content from paywall    
    pre_wiev = compare_element
    content  = soup.find_all( "p", attrs={"class" : "paywall"} )

    # get remaining text
    text_cont = " ".join( [ element.get_text() for element in content ] ) 
    
    try:
        text = " ".join( [ element.get_text() for element in pre_wiev ] ) 
        # get first sentense, since its prewiev
        text = sent_tokenize(text)[0]
        text_cont = text + text_cont
    except Exception as e:
        print(traceback.format_exc())
        print( str(e) ) 

    text_cont = htmltotext( text_cont )
    
    # get article time
    time = soup.find( attrs={"class" : "article-timestamp"} ).get_text()
    
    return text_cont, time

In [12]:
def main_process( category, category_articles_data, sleep_between_url , url, url_server ):
    """ Scraping process. From url get new articles. From articles get content
    
        Args:
            category ([string]) : [Which category will collect from]
            category_articles_data ([pandas.DataFrame]) : [data frame with columns ["url", "title", "text", "date", "category"] ]
            sleep_between_url ([float]) : [sleep time between each url]
            url_server ([string]) : [where send article data]
    """
    
    # check new articles    
    articles_url_name_new = get_urls_from_category( url )
    articles_old_urls = category_articles_data["url"].values
    new_articles_urls = check_new_articles( list( articles_url_name_new.keys() ) , articles_old_urls )
    
    # if appear new article. get content
    for article_url in new_articles_urls:
        
        text_cont, time_art = get_article_content( article_url )
        
        # create new article data
        new_data = {
            "url"      : article_url,
            "title"    : articles_url_name_new[article_url],
            "text"     : text_cont,
            "date"     : time_art,
            "category" : category
        }
        category_articles_data = category_articles_data.append( new_data, ignore_index=True )
        
        # each 10 article pickle
        if category_articles_data.shape[0] % 10 == 0:
            pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 
        # if article counts > 300, save 150        
        if category_articles_data.shape[0] > 300:
            category_articles_data = save_data( category_articles_data, df_save_path )
            pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 
            
        # send data to server
        try:
            send_query( url_server, new_data )
        except:
            pass
        
        time.sleep( sleep_between_url )
    
    # dump after each category since articles can repeat    
    pickle.dump( category_articles_data , open(pickle_obj_path,"wb"))

In [13]:
categorys = [ "/market", "/technology", "/politics", "/markets/economics"]
df_columns = ["url", "title", "text", "date", "category"]

url_server = "http://localhost:9000/"
source_url = "https://www.bloomberg.com"
pickle_obj_path = "bloomberg_data.pkl"
df_save_path = "./Bloomberg/bloomberg_data"

sleep_between_url = 3
sleep_between_cat = 3

In [14]:
category_articles_data = pd.DataFrame( columns = df_columns )

In [ ]:
category_articles_data = pickle.load(open(pickle_obj_path, "rb"))

In [ ]:
while True:

    for category in categorys:

        url = source_url + category
        category_articles_data = pickle.load(open(pickle_obj_path, "rb"))
        
        try:
            main_process( category, category_articles_data, sleep_between_url, url, url_server )
        except:
            change_browser()
        
        time.sleep(sleep_between_cat)

# Save and Load data

In [ ]:
pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 

In [ ]:
category_articles_data = pickle.load(open(pickle_obj_path, "rb"))